# Exporing aviation accidents data

Link to kagge: https://www.kaggle.com/khsamaha/aviation-accident-database-synopses

## Dataset purpose
The declared purpose for publishing the dataset was:
> * Which is the type accident often to happen? Which are the features relevant?
* What is season that there are more accident?
* The amateur have a influence on accident or injury severity?
* Do they take too long to make preliminary reports?
* What do scheme have more accident?
* Where are there more accident? - deprecated
* What do aircraft have more accident? -deprecated
* How do accidents evolve in the time of aviation in the United States?



In [1]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, preprocessing, svm
from sklearn.preprocessing import StandardScaler, Normalizer
import math
import re
import seaborn as sns


In [2]:
df = pd.read_csv('AviationDataEnd2016UP.csv', sep=',', header=0, encoding = 'iso-8859-1')

df.sample(10)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.Flight,Air.Carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Report.Status,Publication.Date
11034,20100526X92845,Accident,ERA10WA281,2010-05-25,"North Bimini, Bahamas",Bahamas,25.700000,-79.265278,MYBS,South Bimini Intl. Airport,...,Personal,NaN,2.0,2.0,NaN,NaN,VMC,NaN,Foreign,07/07/2010
35157,20001211X10310,Accident,FTW98LA264,1998-06-14,"HEMPSTEAD, TX",United States,NaN,NaN,35R,HEMPSTEAD GLIDERPORT,...,Instructional,NaN,0.0,0.0,1.0,0.0,VMC,LANDING,Probable Cause,16/02/2001
50792,20001212X17598,Accident,LAX91LA307,1991-07-11,"EL CERRITO, CA",United States,NaN,NaN,NaN,NaN,...,Instructional,NaN,0.0,1.0,1.0,0.0,VMC,TAKEOFF,Probable Cause,24/03/1993
48176,20001211X15443,Accident,LAX92LA344,1992-08-08,"NOVATO, CA",United States,NaN,NaN,O56,NOVATO,...,Personal,NaN,0.0,0.0,0.0,4.0,VMC,LANDING,Probable Cause,29/09/1993
63278,20001213X34796,Accident,FTW86LA174,1986-09-30,"LAREDO, TX",United States,NaN,NaN,LRD,LAREDO INTL,...,Unknown,NaN,0.0,0.0,0.0,1.0,VMC,TAKEOFF,Probable Cause,NaN
58525,20001213X26206,Accident,DEN88LA148,1988-07-07,"CRAIG, CO",United States,NaN,NaN,CAG,CRAIG-MOFFAT,...,Personal,NaN,0.0,0.0,1.0,1.0,VMC,DESCENT,Probable Cause,11/09/1989
35710,20001211X09660,Accident,FTW98FA156,1998-03-24,"RIO HONDO, TX",United States,NaN,NaN,NaN,NaN,...,Ferry,NaN,2.0,0.0,0.0,0.0,VMC,CRUISE,Probable Cause,16/02/2001
57612,20001213X27070,Accident,LAX89DVA01,1988-10-12,"TRAVER, CA",United States,NaN,NaN,NaN,NaN,...,Aerial Application,NaN,0.0,1.0,0.0,0.0,VMC,MANEUVERING,Probable Cause,29/12/1989
11600,20100120X85827,Accident,CEN10LA103,2010-01-16,"Cedar Rapids, IA",United States,41.884722,-91.710833,CID,Eastern Iowa,...,Personal,NaN,NaN,1.0,2.0,NaN,IMC,APPROACH,Probable Cause,26/05/2010
6754,20120816X14223,Accident,CEN12CA544,2012-08-15,"Kaplan, LA",United States,30.000000,-92.266667,NaN,NaN,...,Aerial Application,NaN,NaN,1.0,NaN,NaN,VMC,MANEUVERING,Probable Cause,27/11/2012


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79293 entries, 0 to 79292
Data columns (total 31 columns):
Event.Id                  79293 non-null object
Investigation.Type        79293 non-null object
Accident.Number           79293 non-null object
Event.Date                79293 non-null object
Location                  79215 non-null object
Country                   78786 non-null object
Latitude                  25751 non-null float64
Longitude                 25742 non-null float64
Airport.Code              44666 non-null object
Airport.Name              47439 non-null object
Injury.Severity           79293 non-null object
Aircraft.Damage           76883 non-null object
Aircraft.Category         22477 non-null object
Registration.Number       76209 non-null object
Make                      79204 non-null object
Model                     79175 non-null object
Amateur.Built             78721 non-null object
Number.of.Engines         75175 non-null float64
Engine.Type             

In [4]:
df.describe()

/opt/conda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Latitude,Longitude,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,25751.000000,25742.000000,75175.000000,55984.000000,53742.000000,54833.000000,66949.000000
mean,37.690421,-93.781061,1.148055,0.814679,0.317703,0.502581,5.790886
std,12.148019,39.243662,0.453847,6.233700,1.372924,2.781994,29.223016
min,-78.016945,-178.676111,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,89.218056,177.557778,18.000000,349.000000,111.000000,380.000000,699.000000


In [5]:
df.count()

Event.Id                  79293
Investigation.Type        79293
Accident.Number           79293
Event.Date                79293
Location                  79215
Country                   78786
Latitude                  25751
Longitude                 25742
Airport.Code              44666
Airport.Name              47439
Injury.Severity           79293
Aircraft.Damage           76883
Aircraft.Category         22477
Registration.Number       76209
Make                      79204
Model                     79175
Amateur.Built             78721
Number.of.Engines         75175
Engine.Type               75919
FAR.Description           22334
Schedule                  11501
Purpose.of.Flight         75399
Air.Carrier                3918
Total.Fatal.Injuries      55984
Total.Serious.Injuries    53742
Total.Minor.Injuries      54833
Total.Uninjured           66949
Weather.Condition         77136
Broad.Phase.of.Flight     73239
Report.Status             79293
Publication.Date          65819
dtype: i

In [6]:
# splitting date field in the components
import datetime

df['Year'] = df['Event.Date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d").year)
df['Month'] = df['Event.Date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d").month)
df['Day'] = df['Event.Date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d").day)


### Looking at some categories
I try to list some unique values in the categories fields to subsequently plot some data distribution over those.

In [7]:
categories = ['Investigation.Type',
             'Aircraft.Damage',
             'Aircraft.Category',
             'Amateur.Built',
             'Number.of.Engines',
             'Engine.Type',
             'FAR.Description',
             'Schedule',
             'Purpose.of.Flight',
             'Weather.Condition',
             'Broad.Phase.of.Flight',
             'Report.Status']

for c in categories:
    print(c , df[c].unique())



Investigation.Type ['Accident' 'Incident']
Aircraft.Damage ['Substantial' 'Destroyed' nan 'Minor']
Aircraft.Category ['Airplane' 'Helicopter' 'Weight-Shift' 'Glider' 'Unknown' 'Balloon'
 'Powered Parachute' 'Ultralight' 'Gyroplane' 'Gyrocraft' nan
 'Powered-Lift' 'Rocket' 'Blimp']
Amateur.Built ['Yes' 'No' nan]
Number.of.Engines [ nan   1.   2.   0.   4.   3.  18.]
Engine.Type ['Reciprocating' nan 'Turbo Prop' 'Turbo Fan' 'Turbo Shaft' 'Unknown'
 'Turbo Jet' 'Electric' 'REC, ELEC' 'None' 'TF, TJ' 'Hybrid Rocket'
 'REC, TJ, TJ' 'REC, TJ, REC, TJ' 'TJ, REC, REC, TJ']
FAR.Description ['Part 91: General Aviation' nan 'Part 135: Air Taxi & Commuter'
 'Public Aircraft' 'Part 121: Air Carrier' 'Unknown'
 'Non-U.S., Non-Commercial' 'Part 137: Agricultural' 'Non-U.S., Commercial'
 'Part 103: Ultralight' 'Part 133: Rotorcraft Ext. Load' 'Public Use'
 'Part 129: Foreign' 'Armed Forces' 'Part 437: Commercial Space Flight'
 'Part 91 Subpart K: Fractional' 'Part 125: 20+ Pax,6000+ lbs'
 'Part 91F: S

In [8]:
for c in categories:
    print('\n', df.groupby(by=c)[c].count().sort_values(ascending=False))
    print('Nulls: ', df[c].isnull().sum())



 Investigation.Type
Accident    76118
Incident     3175
Name: Investigation.Type, dtype: int64
Nulls:  0

 Aircraft.Damage
Substantial    57049
Destroyed      17322
Minor           2512
Name: Aircraft.Damage, dtype: int64
Nulls:  2410

 Aircraft.Category
Airplane             19273
Helicopter            2360
Glider                 381
Balloon                175
Gyrocraft              100
Weight-Shift            66
Powered Parachute       48
Unknown                 32
Ultralight              31
Powered-Lift             5
Blimp                    3
Gyroplane                2
Rocket                   1
Name: Aircraft.Category, dtype: int64
Nulls:  56816

 Amateur.Built
No     71105
Yes     7616
Name: Amateur.Built, dtype: int64
Nulls:  572

 Number.of.Engines
1.0     63082
2.0     10057
0.0      1143
3.0       477
4.0       415
18.0        1
Name: Number.of.Engines, dtype: int64
Nulls:  4118

 Engine.Type
Reciprocating       64598
Turbo Shaft          3305
Turbo Prop           3042
Turbo 

In [9]:
# null damages can't be defined
df[df['Aircraft.Damage'].isnull()]
df['Aircraft.Damage'].fillna('Unknown', inplace=True)

In [10]:
# null countries are outside US
df[df['Country'].isnull()]
df['Country'].fillna('Foreign', inplace=True)

In [11]:
# null categories can't be defined
df[df['Aircraft.Category'].isnull()]
df['Aircraft.Category'].fillna('Unknown', inplace=True)

In [12]:
# null amateur seem to be all commercial planes producers
print(df[df['Amateur.Built'].isnull()]['Make'].unique())
df['Amateur.Built'].fillna('No', inplace=True)


['BOMBARDIER INC' 'Boeing Commercial Airplane Gro' 'BELLANCA' 'AIRBUS'
 'EMBRAER' 'BEECH' 'PIPER' 'OGG RICHARD A' 'HELIO'
 'MCDONNELL DOUGLAS HELICOPTER' 'Quad City' 'LIMING LOUIS L'
 'SONEX LIMITED' 'FLIGHT DESIGN GMBH' 'Dehavilland' 'RANS'
 'AIRBUS INDUSTRIE' 'BELL' 'THE BOEING COMPANY' 'CESSNA'
 'CIRRUS DESIGN CORP' 'Wag-Aero' 'VANS' 'BOEING' 'ROCKWELL' nan 'Boeing'
 'MOONEY' 'BOMBARDIER' 'BUCKER JUNGMEISTER' 'COLUMBIA AIRCRAFT MFG'
 'Trendak' 'REIMS' 'BELL HELICOPTER TEXTRON' 'GRUMMAN AMERICAN AVN. CORP.'
 'SWEARINGEN' 'MX AIRCRAFT LLC' 'SIKORSKY' 'DEHAVILLAND' 'ATR'
 'WORLD AIRCRAFT CO' 'LAKE' 'HILLER' 'EDGE' 'BUCKEYE' 'BOEING COMPANY'
 'SHORT BROTHERS & HARLAND LTD.' 'AEROSPOOL' 'EIPPER' 'FOKKER'
 'ROBINSON HELICOPTER' 'GATES LEARJET' 'GULFSTREAM AEROSPACE'
 'SIAI-MARCHETTI' 'CELIER AVIATION' 'REIMS AVIATION SA'
 'AMATEUR CONSTRUCTION' 'MAULE' 'SOCATA' 'MCDONNELL DOUGLAS' 'AEROPRAKT'
 'ROBINSON HELICOPTER COMPANY' 'AIR TRACTOR' 'EXTRA' 'HOFFIUS STOWE'
 'SUI RY, LAURINEN JANNE, K?

In [23]:

df[df['Make']=='BALLOON WORKS'].fillna(0, inplace=True)
df[df['Make']=='Balloon Works'].fillna(0, inplace=True)
df[df['Make']=='CESSNA'].fillna(1, inplace=True)
df[df['Make']=='PIPER'].fillna(1, inplace=True)

df[df['Number.of.Engines'].isnull()][['Make', 'Model']]
df['Number.of.Engines'] = 0

/opt/conda/lib/python3.5/site-packages/pandas/core/frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [24]:

les = {}

for l in categories:
    print("Labeling ", l)
    les[l] = preprocessing.LabelEncoder()
    les[l].fit(df[l])
    tr = les[l].transform(df[l]) 
    df.loc[:, l + '_feat'] = pd.Series(tr, index=df.index)
    print(les[l].classes_)



Labeling  Investigation.Type
['Accident' 'Incident']
Labeling  Aircraft.Damage
['Destroyed' 'Minor' 'Substantial' 'Unknown']
Labeling  Aircraft.Category
['Airplane' 'Balloon' 'Blimp' 'Glider' 'Gyrocraft' 'Gyroplane' 'Helicopter'
 'Powered Parachute' 'Powered-Lift' 'Rocket' 'Ultralight' 'Unknown'
 'Weight-Shift']
Labeling  Amateur.Built
['No' 'Yes']
Labeling  Number.of.Engines
[0]
Labeling  Engine.Type


TypeError: unorderable types: str() > float()